In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
# import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vp
import model_validation.ciff_sam_results as csr
import model_validation.ciff_sam_plots as csp

%load_ext autoreload
%autoreload 2

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model4
ndbs
Thu Oct 14 15:23:36 PDT 2021


# Load data and set global index columns

In [2]:
results = csr.VivariumResults.cleaned_from_model_spec('4.5.2')
results.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

In [3]:
vp.set_global_index_columns(vp.INDEX_COLUMNS+['x_factor_effect'])
vp.INDEX_COLUMNS

['input_draw', 'scenario', 'x_factor_effect']

# Test new `assert_cause_person_time_equal` function

In [4]:
csr.assert_cause_person_time_equal(results)

# Work on computing incidence rate ratio for X-factor

In [5]:
results.wasting_transition_count

,sex,year,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age,transition,from_state,to_state
0,female,2022,transition_count,29,baseline,1.1,0.0,cat2,covered,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,1.1,0.0,cat1,covered,covered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,1.1,0.0,cat2,covered,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,1.1,0.0,cat1,covered,uncovered,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604796,male,2026,transition_count,946,wasting_treatment,1.5,1629.0,cat2,uncovered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604797,male,2026,transition_count,946,wasting_treatment,1.5,483.0,cat1,uncovered,covered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604798,male,2026,transition_count,946,wasting_treatment,1.5,167.0,cat2,uncovered,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604799,male,2026,transition_count,946,wasting_treatment,1.5,60.0,cat1,uncovered,uncovered,2_to_4,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting


In [6]:
results.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,covered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,uncovered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...
345596,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,16475.270363,cat2,uncovered,covered,2_to_4
345597,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,3419.826146,cat1,uncovered,covered,2_to_4
345598,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,1704.976044,cat2,uncovered,uncovered,2_to_4
345599,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,384.646133,cat1,uncovered,uncovered,2_to_4


# Find the 3 transitions representing wasting state incidence rates

In [7]:
results.wasting_transition_count.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'mild_child_wasting_to_susceptible_to_child_wasting',
       'moderate_acute_malnutrition_to_mild_child_wasting',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition',
       'severe_acute_malnutrition_to_mild_child_wasting',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition',
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [8]:
wasting_incidence_transitions = [
    'susceptible_to_child_wasting_to_mild_child_wasting',
    'mild_child_wasting_to_moderate_acute_malnutrition',
    'moderate_acute_malnutrition_to_severe_acute_malnutrition',
]

# Check that transition counts are 0 for late neonatal

They should be 0 for all age groups under 6 months.

In [11]:
results.wasting_transition_count.query("age == 'late_neonatal'")

,sex,year,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age,transition,from_state,to_state
8,female,2022,transition_count,29,baseline,1.1,0.0,cat2,covered,covered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
9,female,2022,transition_count,29,baseline,1.1,0.0,cat1,covered,covered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
10,female,2022,transition_count,29,baseline,1.1,0.0,cat2,covered,uncovered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
11,female,2022,transition_count,29,baseline,1.1,0.0,cat1,covered,uncovered,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition,mild_child_wasting,moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604764,male,2026,transition_count,946,wasting_treatment,1.5,0.0,cat2,uncovered,covered,late_neonatal,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604765,male,2026,transition_count,946,wasting_treatment,1.5,0.0,cat1,uncovered,covered,late_neonatal,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604766,male,2026,transition_count,946,wasting_treatment,1.5,0.0,cat2,uncovered,uncovered,late_neonatal,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting
604767,male,2026,transition_count,946,wasting_treatment,1.5,0.0,cat1,uncovered,uncovered,late_neonatal,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,mild_child_wasting


In [23]:
(results.wasting_transition_count.query("age == 'late_neonatal'").value==0).all()

True

# Get age group bins for over 6mo and under 6mo

In [16]:
csr.get_age_group_bins('6-11_months', 'all_ages')

(['early_neonatal', 'late_neonatal', '1-5_months']
 Categories (7, object): ['early_neonatal' < 'late_neonatal' < '1-5_months' < '6-11_months' < '12_to_23_months' < '2_to_4' < 'all_ages'],
 ['6-11_months', '12_to_23_months', '2_to_4']
 Categories (7, object): ['early_neonatal' < 'late_neonatal' < '1-5_months' < '6-11_months' < '12_to_23_months' < '2_to_4' < 'all_ages'],
 ['all_ages']
 Categories (7, object): ['early_neonatal' < 'late_neonatal' < '1-5_months' < '6-11_months' < '12_to_23_months' < '2_to_4' < 'all_ages'])

In [15]:
under_6mo, over_6mo, all_ages = map(list, csr.get_age_group_bins('6-11_months', 'all_ages'))
print(under_6mo, over_6mo, all_ages)

['early_neonatal', 'late_neonatal', '1-5_months'] ['6-11_months', '12_to_23_months', '2_to_4'] ['all_ages']


# Test `get_transition_rates` function after refactoring

In [18]:
transition_rates_by_age_year = csr.get_transition_rates(results, 'wasting', ['age', 'year'])
transition_rates_by_age_year

,age,year,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,1-5_months,2022,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,0.0,transition_count,state_person_time,1
1,1-5_months,2022,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.0,transition_count,state_person_time,1
2,1-5_months,2022,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,0.0,transition_count,state_person_time,1
3,1-5_months,2022,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.0,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...
37796,late_neonatal,2026,susceptible_to_child_wasting,946,wasting_treatment,1.2,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1
37797,late_neonatal,2026,susceptible_to_child_wasting,946,wasting_treatment,1.3,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1
37798,late_neonatal,2026,susceptible_to_child_wasting,946,wasting_treatment,1.4,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1
37799,late_neonatal,2026,susceptible_to_child_wasting,946,wasting_treatment,1.5,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1


In [26]:
transition_rates_by_age_year.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'mild_child_wasting_to_susceptible_to_child_wasting',
       'moderate_acute_malnutrition_to_mild_child_wasting',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition',
       'severe_acute_malnutrition_to_mild_child_wasting',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition',
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [29]:
transition_rates_by_age_year.isna().any().any()

False

In [19]:
transition_rates_by_age_year_x_factor = csr.get_transition_rates(results, 'wasting', ['age', 'year', 'x_factor'])
transition_rates_by_age_year_x_factor

,age,year,x_factor,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,1-5_months,2022,cat1,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,0.0,transition_count,state_person_time,1
1,1-5_months,2022,cat1,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.0,transition_count,state_person_time,1
2,1-5_months,2022,cat1,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,0.0,transition_count,state_person_time,1
3,1-5_months,2022,cat1,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.0,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75596,late_neonatal,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.2,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1
75597,late_neonatal,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.3,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1
75598,late_neonatal,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.4,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1
75599,late_neonatal,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.5,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.0,transition_count,state_person_time,1


In [31]:
transition_rates_by_age_year_x_factor.isna().any().any()

False

In [20]:
transition_rates_by_age_year_x_factor_filtered = csr.get_transition_rates(
    results,
    'wasting',
    strata=['age', 'year', 'x_factor'],
    transition_query=f"age in {over_6mo}",
    person_time_query=f"age in {over_6mo}",
)
transition_rates_by_age_year_x_factor_filtered

,age,year,x_factor,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.023727,transition_count,state_person_time,1
1,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.327570,transition_count,state_person_time,1
2,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.250496,transition_count,state_person_time,1
3,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_susceptible_to_child_was...,susceptible_to_child_wasting,0.325650,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37796,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.2,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.109266,transition_count,state_person_time,1
37797,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.3,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.106740,transition_count,state_person_time,1
37798,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.4,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105301,transition_count,state_person_time,1
37799,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.5,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.104250,transition_count,state_person_time,1


In [32]:
transition_rates_by_age_year_x_factor_filtered.isna().any().any()

False

# See if the way I'm thinking about the X-factor wasting incidence ratio makes sense

## I'm getting NaN's because the numerator is filtered to transitions representing incidence rates, whereas the denominator still includes person-time in the SAM category, from which there is no incidence

In [22]:
incidence_rates_by_age_year_x_factor_filtered = csr.get_transition_rates(
    results,
    'wasting',
    strata=['age', 'year', 'x_factor'],
    transition_query=f"age in {over_6mo} and transition in {wasting_incidence_transitions}",
    person_time_query=f"age in {over_6mo}"
)
incidence_rates_by_age_year_x_factor_filtered

,age,year,x_factor,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.023727,transition_count,state_person_time,1
1,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.250496,transition_count,state_person_time,1
2,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.3,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.426161,transition_count,state_person_time,1
3,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.4,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.577621,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21596,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.2,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.109266,transition_count,state_person_time,1
21597,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.3,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.106740,transition_count,state_person_time,1
21598,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.4,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105301,transition_count,state_person_time,1
21599,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.5,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.104250,transition_count,state_person_time,1


In [24]:
# Why are there NaNs???
incidence_rates_by_age_year_x_factor_filtered.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition', nan,
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [25]:
incidence_rates_by_age_year_x_factor_filtered.query("transition != transition")

,age,year,x_factor,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier
360,12_to_23_months,2022,cat1,severe_acute_malnutrition,29,baseline,1.1,NaN,NaN,NaN,transition_count,state_person_time,1
361,12_to_23_months,2022,cat1,severe_acute_malnutrition,29,baseline,1.2,NaN,NaN,NaN,transition_count,state_person_time,1
362,12_to_23_months,2022,cat1,severe_acute_malnutrition,29,baseline,1.3,NaN,NaN,NaN,transition_count,state_person_time,1
363,12_to_23_months,2022,cat1,severe_acute_malnutrition,29,baseline,1.4,NaN,NaN,NaN,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21416,6-11_months,2026,cat2,severe_acute_malnutrition,946,wasting_treatment,1.2,NaN,NaN,NaN,transition_count,state_person_time,1
21417,6-11_months,2026,cat2,severe_acute_malnutrition,946,wasting_treatment,1.3,NaN,NaN,NaN,transition_count,state_person_time,1
21418,6-11_months,2026,cat2,severe_acute_malnutrition,946,wasting_treatment,1.4,NaN,NaN,NaN,transition_count,state_person_time,1
21419,6-11_months,2026,cat2,severe_acute_malnutrition,946,wasting_treatment,1.5,NaN,NaN,NaN,transition_count,state_person_time,1


In [33]:
incidence_rates_by_age_year_x_factor_filtered.isna().any().any()

True

In [34]:
incidence_rates_by_age_year_x_factor_filtered.query("transition != transition").from_state.unique()

array(['severe_acute_malnutrition'], dtype=object)

# Try to fix it...

In [36]:
results.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,x_factor_effect,value,x_factor,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,covered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat2,covered,uncovered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,1.1,0.000000,cat1,covered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...
345596,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,16475.270363,cat2,uncovered,covered,2_to_4
345597,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,3419.826146,cat1,uncovered,covered,2_to_4
345598,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,1704.976044,cat2,uncovered,uncovered,2_to_4
345599,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,1.5,384.646133,cat1,uncovered,uncovered,2_to_4


In [41]:
strata = ['age', 'year', 'x_factor']
#for colname in strata:
colname = 'age'
mask = results.wasting_state_person_time[colname].isin(incidence_rates_by_age_year_x_factor_filtered[colname])
results.wasting_state_person_time.loc[mask, colname].unique()

array(['6-11_months', '12_to_23_months', '2_to_4'], dtype=object)

In [43]:
numerator = incidence_rates_by_age_year_x_factor_filtered
# denominator = results.wasting_state_person_time
stratum_lists = {colname: tuple(numerator[colname].unique()) for colname in strata}
person_time_query = " and ".join(f"({colname} in {stratum_list})" for colname, stratum_list in stratum_lists.items())
print(stratum_value_lists)
print(person_time_query)

[('12_to_23_months', '2_to_4', '6-11_months'), ('2022', '2023', '2024', '2025', '2026'), ('cat1', 'cat2')]
(age in ('12_to_23_months', '2_to_4', '6-11_months')) and (year in ('2022', '2023', '2024', '2025', '2026')) and (x_factor in ('cat1', 'cat2'))


In [45]:
kwargs = {}
kwargs['numerator_broadcast'] = vp.list_columns(
        'transition', 'to_state', kwargs.get('numerator_broadcast'), default=[])
kwargs

{'numerator_broadcast': ['transition', 'to_state']}

In [46]:
kwargs['numerator_broadcast'] = vp.list_columns(
        'x_factor', kwargs.get('numerator_broadcast'), default=[])
kwargs

{'numerator_broadcast': ['x_factor', 'transition', 'to_state']}

# Test new version of get_transition_rates

Looks like it works.

In [47]:
incidence_rates_by_age_year_x_factor_filtered2 = csr.get_transition_rates(
    results,
    'wasting',
    strata=['age', 'year', 'x_factor'],
    prefilter_query=f"age in {over_6mo} and transition in {wasting_incidence_transitions}",
)
incidence_rates_by_age_year_x_factor_filtered2

,age,year,x_factor,from_state,input_draw,scenario,x_factor_effect,transition,to_state,value,numerator_measure,denominator_measure,multiplier
0,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.1,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.023727,transition_count,state_person_time,1
1,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.2,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.250496,transition_count,state_person_time,1
2,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.3,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.426161,transition_count,state_person_time,1
3,12_to_23_months,2022,cat1,mild_child_wasting,29,baseline,1.4,mild_child_wasting_to_moderate_acute_malnutrition,moderate_acute_malnutrition,3.577621,transition_count,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16196,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.2,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.109266,transition_count,state_person_time,1
16197,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.3,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.106740,transition_count,state_person_time,1
16198,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.4,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.105301,transition_count,state_person_time,1
16199,6-11_months,2026,cat2,susceptible_to_child_wasting,946,wasting_treatment,1.5,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.104250,transition_count,state_person_time,1


In [49]:
incidence_rates_by_age_year_x_factor_filtered2.isna().any().any()

False

In [50]:
incidence_rates_by_age_year_x_factor_filtered2.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition',
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [63]:
n = len(incidence_rates_by_age_year_x_factor_filtered2)
vp.compare_values(
    incidence_rates_by_age_year_x_factor_filtered.loc[:n-1,:],
    incidence_rates_by_age_year_x_factor_filtered2
)

age,denominator_measure,from_state,input_draw,multiplier,numerator_measure,scenario,to_state,transition,x_factor,x_factor_effect,year


In [64]:
vp.assert_values_equal(
    incidence_rates_by_age_year_x_factor_filtered.loc[:n-1,:],
    incidence_rates_by_age_year_x_factor_filtered2
)

In [65]:
n

16200

# Try computing incidence rate ratio

Lookin' good

In [70]:
csr.get_x_factor_incidence_ratio(results, ['year', 'age'])

,year,age,transition,input_draw,scenario,x_factor_effect,value,numerator_measure,denominator_measure,multiplier
0,2022,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.1,1.112005,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
1,2022,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.2,1.216690,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
2,2022,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.3,1.299578,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
3,2022,12_to_23_months,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.4,1.373513,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
...,...,...,...,...,...,...,...,...,...,...
8096,2026,6-11_months,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.2,1.154112,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
8097,2026,6-11_months,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.3,1.278448,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
8098,2026,6-11_months,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.4,1.329408,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
8099,2026,6-11_months,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.5,1.412776,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1


In [71]:
csr.get_x_factor_incidence_ratio(results, ['year'])

,year,transition,input_draw,scenario,x_factor_effect,value,numerator_measure,denominator_measure,multiplier
0,2022,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.1,1.109096,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
1,2022,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.2,1.212432,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
2,2022,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.3,1.305246,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
3,2022,mild_child_wasting_to_moderate_acute_malnutrition,29,baseline,1.4,1.392815,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
...,...,...,...,...,...,...,...,...,...
2696,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.2,1.170304,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
2697,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.3,1.273938,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
2698,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.4,1.372248,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1
2699,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1.5,1.448412,incidence_rate_among_x_factor_cat1,incidence_rate_among_x_factor_cat2,1


# Potential template for generic relative risk function

Note: It is possible to design the function so that `prefilter_query` can be used to filter to specific desired outcomes (e.g. specific wasting or stunting states, specific transitions, or deaths due to a specific cause), like I did above to filter to only the wasting incidence transitions, but this is unnecessary because the filtering can be done afterwards.

I think it will simplify the code and help clarify thinking about the problems to reserve `prefilter_query` for filtering things that _can't_ be done after computing the ratio, namely filtering a stratm before aggregating over that stratum. It could also be used for filtering scenarios, e.g. to avoid NaNs from divide-by-zero, but that can also be done after the division.

In particular, I _think_ it should be sufficient to only allow queries that can be passed to _both_ the numerator and denominator, rather than having to handle filtering the numerator and denominator separately because they have differeent columns. It seems like any other kind of query should be handlable by broadcasting and post-filtering. The downside is doing extra computation and returning tables that are larger than necessary, which could potentially be confusing. But I think the performance difference should be negligible for our purposes, and I think the benefits of enforcing more structure outweigh the benefits of allowing more flexibility.

In [ ]:
def get_relative_risk(data, measure, table_entity, strata, factor, reference_category, prefilter_query=None):
    """
    `measure` could be one of 'prevalence', 'incidence', 'mortality', 'remission' (or generically, 'rate').
        Maybe 'prevalence', 'transition_rate', or 'mortality_rate' since each of these has a different type
        of table for the numerator (person time, transition_count, or deaths).
    `table_entity` or `outcome_variable`??? is passed to either get_transition_rates or get_prevalence,
        and represents the outcome for which we want to compute the relative risk (e.g. 'stunting_state',
        'wasting_state', or a stratification variable for measure=='prevalence', or 'wasting' or 'cause' for 
        measure=='transition_rate', or 'cause'??? or 'death'??? or None??? or cause_name???
        for measure=='mortality_rate').
        Note that `table_entity` may be sort of a "meta-description" of the outcome we're interested in,
        with the actual outcome being one or more items described by this variable (e.g. the specific
        stunting or wasting categories, specific wasting state or cause state transitions, or deaths from
        a specific cause).
    `factor` is the (risk) factor for which we want to compute the relative risk (e.g. x_factor, sq_lns, 
    stunting_state, wasting_state).
    `reference_category` is the factor category to put in the denominator to use as a reference for computing
    relative risks (e.g. the TMREL). The numerator should be broadcast over all remaining categories. (Note: In order
    to do that, I will have to employ a similar strategy to what I did in the `difference` function, and add the
    factor category column to the index with different names in the numerator and denominator.)
    """
    pass